# Resume Recommender

Experimenting with a way to handle (potentially) long inputs that can't be truncated - recommending resumes with retrieve-and-rank!

In [69]:
from typing import *
import os.path
import re
import textwrap
from uuid import uuid4
import zipfile
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from cleantext import clean
import dill

[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Outline Of Approach
I'll be using a [retrieve and rank approach](https://www.sbert.net/examples/applications/retrieve_rerank/README.html#retrieve-re-rank).  The basic idea is to reduce what could be a huge list of possibilities with an initial coarse filtration (the "retrieval"), then apply a fine filtration (the "rank") to return the best possible results:

* From the universe of available candidates, choose an initial list of candidates with a (relatively) fast and performant technique.  I'll be using the excellent [SentenceTransformer](https://www.sbert.net/docs/quickstart.html#quickstart).
* From this subset, re-rank the candidates with a more sophisticated technique.  Here I'll use sentence-transformers' [CrossEncoder](https://www.sbert.net/examples/applications/cross-encoder/README.html), which is basically just the standard transformer architecture we all know and love.

That's the high level, but how will I handle lengthy documents?  I can't just truncate at 256/512 tokens, because there's a good chance that I'll miss the most relevant parts of the resume.  That leaves me with using a sliding window to extract subsets from each document.  I usually get fairly decent results just splitting on whitespace, but here I'm mainly interested in trying out [NLTK's sent_tokenize](https://www.nltk.org/api/nltk.tokenize.html) function.

So here's the full process:

1. **Tokenize** : split each document into a list of sentences.
2. **Retrieval** : for a given query, run a semantic search against each tokenized resume (list of sentences).  The relevance of a given resume to the query is then the maximum score (dot product) between the query and the resume's sentences, and the most relevant resumes are the top N resumes by score.
3. **Ranking** : for each of the top N resumes found in Step 2, compute the [score](https://www.sbert.net/docs/package_reference/cross_encoder.html#sentence_transformers.cross_encoder.CrossEncoder.predict) between a query string and the most relevant sentence in the resume.  Re-order the top N resumes.

## Data
Here I'll use a set of resumes from [florex's resume_corpus](https://github.com/florex/resume_corpus).  There are several thousand resumes in the corpus, so rather than extract to disk I'll just read them directly from the archive.

### References
Jiechieu, K.F.F., Tsopze, N. Skills prediction based on multi-label resume classification using CNN with model predictions explanation. Neural Comput & Applic (2020). https://doi.org/10.1007/s00521-020-05302-x

In [42]:
data_archive = './data/resumes_corpus.zip'  # https://github.com/florex/resume_corpus/blob/master/resumes_corpus.zip


def read_resume_zip(resume_zip_file: AnyStr, resume_extension: AnyStr = '.txt') -> Dict[AnyStr, Any]:
    """
    Directly reads files with a given filename extension.  Files are tokenized into sentences.
    resume_zip_file: path to the ZIP file
    resume_extension: extension of files to read, defaults to '.txt'.
    
    Returns a dict where keys are unique IDs (uuid4) of the form
    {uuid_str: {'sentences': [list of sentences], 'embeddings': None}}
    """
    resumes = dict()
    with zipfile.ZipFile(data_archive, 'r') as datazip:
        resume_files = [file for file in datazip.namelist() if file.endswith(resume_extension)]    
        for file in resume_files:
            with datazip.open(file) as fidin:
                key = str(uuid4())
                resumes[key] = dict()
                resumes[key]['sentences'] = sent_tokenize(clean_text(fidin.read()))
                resumes[key]['embeddings'] = None  # Placeholder for encoding
        print("Found {0} file(s)".format(len(resume_files)))
    return resumes

In [39]:
TAG_RE = re.compile('<.*?>')  # Basic regex to match tags

def clean_text(txt: AnyStr, remove_tags: bool = True) -> AnyStr:
    """
    Cleans up text - fixing Unicode, normalizing line breaks, and optionally stripping tags.
    txt: text to clean
    remove_tags: if True (default), try to strip tags from the cleaned text.  Otherwise leave as-is.
    
    Returns the (hopefully) cleaner text.
    """
    if txt and len(txt) > 0:
        cleaned = clean(
            txt,
            fix_unicode=True,               # fix various unicode errors
            to_ascii=True,                  # transliterate to closest ASCII representation
            lower=False,                     # lowercase text
            no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them
            no_urls=False,                  # replace all URLs with a special token
            no_emails=False,                # replace all email addresses with a special token
            no_phone_numbers=False,         # replace all phone numbers with a special token
            no_numbers=False,               # replace all numbers with a special token
            no_digits=False,                # replace all digits with a special token
            no_currency_symbols=False,      # replace all currency symbols with a special token
            no_punct=False,                 # remove punctuations
            replace_with_punct="",          # instead of removing punctuations you may replace them
            replace_with_url="<URL>",
            replace_with_email="<EMAIL>",
            replace_with_phone_number="<PHONE>",
            replace_with_number="<NUMBER>",
            replace_with_digit="0",
            replace_with_currency_symbol="<CUR>",
            lang="en"                       # set to 'de' for German special handling
        )
        if remove_tags:
            cleaned = re.sub(TAG_RE, '', cleaned)
    else:
        cleaned = txt
    return cleaned

In [3]:
# Bi-encoder - retrieval model
# Full list of options: https://www.sbert.net/docs/pretrained_models.html
# 'multi-qa-MiniLM-L6-cos-v1' with a maximum sequence length of 256 seems to work well
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens

# Cross-encoder - ranking model
# Full list of options: https://www.sbert.net/docs/pretrained-models/ce-msmarco.html#models-performance
# Fairly good results w. 'cross-encoder/ms-marco-TinyBERT-L-2-v2', very fast doc processing
cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2')

Downloading: 100%|██████████| 787/787 [00:00<00:00, 1.15MB/s]
Downloading: 100%|██████████| 17.6M/17.6M [00:00<00:00, 91.5MB/s]
Downloading: 100%|██████████| 525/525 [00:00<00:00, 808kB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 50.9MB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 170kB/s]


In [9]:
def encode_resumes(
    resumes: Dict[AnyStr, Any], 
    bi_encoder: SentenceTransformer) -> Dict[AnyStr, Any]:
    """
    Encodes a set of documents with a SentenceTransformer model.
    
    resumes: set of documents to encode
    bi_encoder: SentenceTransformer model used to encode the documents
    
    Returns a dict of documents with the embeddings field populated.
    """
    for fid in resumes.keys():
        resumes[fid]['embeddings'] = bi_encoder.encode(
            resumes[fid]['sentences'], 
            convert_to_tensor=True, 
            show_progress_bar=False
        )
    return resumes


def read_encode_resumes(
    resume_zip_file: AnyStr,
    bi_encoder: SentenceTransformer) -> Dict[AnyStr, Any]:
    """
    Convenience function - reads a ZIP and generates embeddings in a single call.
    
    resume_zip_file: path to the ZIP file
    bi_encoder: SentenceTransformer model used to encode the documents
    
    Returns a dict of documents with the embeddings field populated.
    """
    
    resumes = read_resume_zip(resume_zip_file)
    return encode_resumes(resumes=resumes, bi_encoder=bi_encoder)


def load_resume_pickle(pkl_file: AnyStr) -> Any:
    """
    Deserializes a set of documents
    
    pkl_file: name of file from which to load object
    
    Returns deserialized object.
    """
    with open(pkl_file, 'rb') as fidin:
        return dill.load(fidin)


def save_resume_pickle(resumes: Any, pkl_file: AnyStr):
    """
    Serializes an object.
    
    resumes: object to serialize
    pkl_file: name of file to which to save object
    """
    with open(pkl_file, 'wb') as fidout:
        dill.dump(resumes, fidout)

In [10]:
# Run this cell if you need embeddings...
resumes = read_encode_resumes(resume_zip_file=data_archive, bi_encoder=bi_encoder)
save_resume_pickle(resumes=resumes, pkl_file='./data/resumes.dpkl')

Found 29783 file(s)


In [11]:
# ...otherwise it's much faster to just load from storage
resumes = load_resume_pickle('./data/resumes.dpkl')

With the data prep done I can finally get to the good part.  Here I'll plan on casting a fairly wide net with retrieval, on the assumption that I might have a few spurious recommendations from a fast and coarse initial filtering.  The main thing is to reduce the universe of candidates for the ranking, which often uses a complicated (slow) model.

In [49]:
def retrieve_candidates(
    candidate_query: AnyStr,
    resumes: Dict[AnyStr, Any],
    bi_encoder: SentenceTransformer,
    n_candidates: int = 10) -> List[Dict[AnyStr, Any]]:
    """
    Retrieves the most relevant documents from a list of candidates.
    
    candidate_query: free-form text query
    resumes: universe of candidates
    bi_encoder: SentenceTransformer model used to encode the documents
    n_candidates: number of candidates to return (defaults to 10).
    
    Returns a list of the top n_candidates.
    """
    question_embedding = bi_encoder.encode(candidate_query, convert_to_tensor=True).cuda()
    results = list()
    for resid in resumes.keys():
        best_sentence_hit = util.semantic_search(
            question_embedding, 
            resumes[resid]['embeddings'], 
            top_k=1, 
            score_function=util.dot_score
        )[0]
        results.append({
            'resume_id': resid,
            'score': best_sentence_hit[0]['score'],
            'sentence_id': best_sentence_hit[0]['corpus_id'],
            'sentence': resumes[resid]['sentences'][best_sentence_hit[0]['corpus_id']],
            'sentences': resumes[resid]['sentences']
        })
    return sorted(results, key=lambda el: el['score'], reverse=True)[:n_candidates]


def rank_candidates(
    candidate_query: AnyStr,
    candidates: List[Dict[AnyStr, Any]],
    cross_encoder: CrossEncoder,
    n_candidates: int = None) -> List[Dict[AnyStr, Any]]:
    """
    Re-ranks a list of candidates by their relevance to a query.
    
    candidate_query: free-form text query
    candidates: universe of candidates
    cross_encoder: CrossEncoder model used to rank the candidates
    n_candidates: number of candidates to return.  If None (default), returns the entire list of candidates.
    
    Returns a list of the top n_candidates.
    """
    if not n_candidates:
        n_candidates = len(candidates)
    results = list(candidates)
    cross_inp = [[candidate_query, candidate['sentence']] for candidate in candidates]
    cross_scores = cross_encoder.predict(cross_inp)
    for i in range(len(results)):
        results[i]['cross_score'] = cross_scores[i]
    return sorted(results, key=lambda el: el['cross_score'], reverse=True)[:n_candidates]

In [50]:
# Retrieval

query = 'SQL Server Database Administrator in Wilmington Delaware'
num_candidates = 5

retrieved_candidates = retrieve_candidates(
    candidate_query=query,
    resumes=resumes,
    bi_encoder=bi_encoder,
    n_candidates=num_candidates * 5  # Cast a wide initial net, we'll down select in ranking
)

retrieved_candidates

[{'resume_id': '112916bf-1c08-43bb-b62a-9d9e2930450a',
  'score': 0.7458795309066772,
  'sentence_id': 3,
  'sentence': 'Database Administrator using SQL Server.',
  'sentences': ["b'Software Developer Software Developer Software Developer - Qinetix Group LLC San Juan, PR Work Experience Software Developer Qinetix Group LLC - Guaynabo, PR April 2018 to Present Full Stack Web developer using Symphony (PHP framework).",
   'Software Engineer BeaconMD Technology - Guaynabo, PR April 2017 to February 2018 Full Stack Mobile Developer using NodeJS and React Native.',
   'Software Developer AON Hewitt - San Juan, PR July 2016 to January 2017 Backend Web Developer using ASP.NET ?',
   'Database Administrator using SQL Server.',
   "Education BS of Computer Science in Computer Science Universidad Politecnica - San Juan, PR July 2011 to May 2017 Skills C#, C++, DJANGO, Git, JAVASCRIPT, MYSQL, PHP, CSS, HTML 5, Jquery Links https://juan-github-page.herokuapp.com'"]},
 {'resume_id': 'd015ba48-3dcb

In [52]:
# (Re-)Ranking

ranked_top_candidates = rank_candidates(
    candidate_query=query,
    candidates=retrieved_candidates,
    cross_encoder=cross_encoder,
    n_candidates=num_candidates
)

ranked_top_candidates

[{'resume_id': 'd015ba48-3dcb-4f79-bd7e-13afbe07fa8b',
  'score': 0.7452083826065063,
  'sentence_id': 52,
  'sentence': 'Senior Database Administrator - Technical lead Wilmington Trust Company - Wilmington, DE 2001 to 2010 for a team of seven database administrators that are responsible for administrating over 100 SQL Servers holding approximately 1100 unique user databases.',
  'sentences': ["b'Database Consultant Database Consultant Database Consultant West Chester, PA A challenging and rewarding position as a Database Administrator\\Architect.",
   'Work Experience Database Consultant Anexinet 2017 to 2019 KTMC - Worked on a system to track the processing of claims and settlements for a securities fraud claims and settlements application.',
   'Re-factored an existing transaction\\warehouse hybrid database to conform to database best practices and to add flexibility for new application features that will be created in the near future.',
   'Create new and re factored existing, ETL 

Now I can bring it all back together - given a query, return a list of the top N matches.

In [64]:
def suggest_candidates(
    job_query: AnyStr,
    resumes: Dict[AnyStr, Any],
    bi_encoder: SentenceTransformer,
    cross_encoder: CrossEncoder,
    n_candidates: int = 3) -> List[Dict[AnyStr, Any]]:
    """
    Recommends a list of candidate resumes based on their relevance to a free-form text query.
    
    job_query: free-form text query
    resumes: universe of candidates
    bi_encoder: SentenceTransformer model used to encode the documents
    cross_encoder: CrossEncoder model used to rank the candidates
    n_candidates: number of candidates to return (defaults to 10).
    """
    retrieved_candidates = retrieve_candidates(
        candidate_query=job_query,
        resumes=resumes,
        bi_encoder=bi_encoder,
        n_candidates=num_candidates * 10
    )
    ranked_top_candidates = rank_candidates(
        candidate_query=query,
        candidates=retrieved_candidates,
        cross_encoder=cross_encoder,
        n_candidates=num_candidates
    )
    return [
        {'resume_id': candidate['resume_id'],
         'relevant_snippet': candidate['sentence'],
         'resume_text': ' '.join(candidate['sentences'])
        } for candidate in ranked_top_candidates
    ]

In [75]:
for candidate in suggest_candidates(
    job_query='.NET Software Developer in Florida or Georgia',
    resumes=resumes,
    bi_encoder=bi_encoder,
    cross_encoder=cross_encoder,
    n_candidates=5
):
    print("Resume ID: {0}".format(candidate['resume_id']))
    print("Relevant snippet:\n'{0}'".format(textwrap.fill(candidate['relevant_snippet']))),
    print("\n")
    print("{0}...".format(textwrap.shorten(candidate['resume_text'], width=500)))
    print(25 * '-')
    print()

Resume ID: 7ea62c15-aeab-4ecb-844e-f00b41956e95
Relevant snippet:
'Marietta, GA Professional .Net C#/VB.NET SQL Server web developer.'


b'Senior Developer Senior Developer .NET Senior Software Developer. Marietta, GA Professional .Net C#/VB.NET SQL Server web developer. Work Experience Senior Developer Georgia Department of Health March 2013 to Present Developed several ASP.Net based Web applications using C# and VB.Net and SQL Server with heavy use of HTML, CSS, Javascript, JQuery and Bootstrap. Worked with developer and QA teams to resolve the existing issues. Made ad-hoc enhancements to the different webpages. Helped [...]...
-------------------------

Resume ID: 288c11ea-45e6-485f-8de9-e8e6efd50931
Relevant snippet:
'b'Sr.Net Developer Sr.Net Developer Sr.Net Developer - Norfolk
Southern Corp Trenton, NJ Over 12+ years of experience in Web and
Client/Server applications using Microsoft .NET Technology, SQL
Server, Oracle, C#,ASP.NET, ADO.NET, VB.NET,WCF, ASMX, XML and java
script.